**CHAPTER 1 DATA COLLECTION**

**1.1 Extract the data from MySQL database**

1.1.1 Install PyMySQL


In [2]:
! pip install pymysql

1.1.2 Config DB credential: Use config connect to database

In [4]:
from google.colab import userdata
class Config:
  MYSQL_HOST = userdata.get("MYSQL_HOST")
  MYSQL_PORT = userdata.get("MYSQL_PORT")
  MYSQL_USER = userdata.get("MYSQL_USER")
  MYSQL_PASSWORD = userdata.get("MYSQL_PASSWORD")
  MYSQL_DB = 'r2de3'
  MYSQL_CHARSET = 'utf8mb4'

1.1.3 Connect to DB

In [5]:
import sqlalchemy
engine = sqlalchemy.create_engine(
    "mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(
        user=Config.MYSQL_USER,
        password=Config.MYSQL_PASSWORD,
        host=Config.MYSQL_HOST,
        port=Config.MYSQL_PORT,
        db=Config.MYSQL_DB,
    )
)

1.1.4 Show Tables

In [6]:
with engine.connect() as connection:
    result = connection.execute(sqlalchemy.text(f"show tables;")).fetchall()
result


[('customer',), ('product',), ('transaction',)]

1.1.5 Describe Tables

In [7]:
with engine.connect() as connection:
    desc_transaction = connection.execute(sqlalchemy.text(f"describe transaction")).fetchall()
    desc_customer = connection.execute(sqlalchemy.text(f"describe customer")).fetchall()
    desc_product = connection.execute(sqlalchemy.text(f"describe product")).fetchall()
print("== transaction ==")
print(desc_transaction)
print("== customer ==")
print(desc_customer)
print("== product ==")
print(desc_product)

== transaction ==
[('TransactionNo', 'text', 'YES', '', None, ''), ('Date', 'datetime', 'YES', '', None, ''), ('ProductNo', 'text', 'YES', '', None, ''), ('Price', 'double', 'YES', '', None, ''), ('Quantity', 'bigint(20)', 'YES', '', None, ''), ('CustomerNo', 'double', 'YES', '', None, '')]
== customer ==
[('CustomerNo', 'double', 'YES', '', None, ''), ('Country', 'text', 'YES', '', None, ''), ('Name', 'text', 'YES', '', None, '')]
== product ==
[('ProductNo', 'text', 'YES', '', None, ''), ('ProductName', 'text', 'YES', '', None, '')]


1.1.6 Info: Table and Schema of data

Tables:
*   r2de3.transaction - data of transaction
*   r2de3.customer - data of customer
*   r2de3.product - data of product


  

  




1.1.7 Query Table (Method 1: sqlalchemy)

In [8]:
with engine.connect() as connection:
  product_result = connection.execute(sqlalchemy.text("SELECT * FROM r2de3.product;")).fetchall()
print("number of rows: ", len(product_result))

number of rows:  3768


1.1.7.1 Convert data to Pandas

In [9]:
import pandas as pd
product = pd.DataFrame(product_result)
product = product.set_index("ProductNo")

1.1.8 Query Table (Method 2: Pandas)

In [10]:
customer = pd.read_sql("SELECT * FROM r2de3.customer", engine)

customer

,CustomerNo,Country,Name
0,17490.0,United Kingdom,Sara Griffin
1,13069.0,United Kingdom,Michael Holt
2,12433.0,Norway,Kelli Sandoval
3,13426.0,United Kingdom,Dalton Graves
4,17364.0,United Kingdom,Michelle James
...,...,...,...
4734,16274.0,United Kingdom,Megan Young
4735,14142.0,United Kingdom,Luke Williams
4736,13065.0,United Kingdom,Lisa Jones
4737,18011.0,United Kingdom,Kelly Jenkins


1.1.8.1 Query for Select data from table r2de3.transaction

In [11]:
transaction = pd.read_sql("SELECT * FROM r2de3.transaction", engine)

transaction

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo
0,581482,2024-05-10,22485,21.47,12,17490.0
1,581475,2024-05-10,22596,10.65,36,13069.0
2,581475,2024-05-10,23235,11.53,12,13069.0
3,581475,2024-05-10,23272,10.65,12,13069.0
4,581475,2024-05-10,23239,11.94,6,13069.0
...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0
536346,C536548,2023-05-03,21218,14.09,-3,12472.0
536347,C536548,2023-05-03,20957,11.74,-1,12472.0
536348,C536548,2023-05-03,22580,16.35,-4,12472.0


1.1.9 Join tables: product & customer & transaction

Key for merge the table is:
*   transaction: ProductNo, CustomerNo
*   product: ProductNo
*   customer: CustomerNo

In [12]:
merged_transaction = transaction.merge(product, how="left", left_on="ProductNo", right_on="ProductNo").merge(customer, how="left", left_on="CustomerNo", right_on="CustomerNo")

merged_transaction

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt
...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena


**1.2 Extract the conversion rate data from API withRequests**

1.2.1 Package requests use for REST API


In [13]:
import requests

1.2.2 Requests library Call API (HTTP GET) for conversion rate

In [15]:
url = "https://r2de3-currency-api-vmftiryt6q-as.a.run.app/gbp_thb"
r = requests.get(url)
result_conversion_rate = r.json()

result_conversion_rate

[{'date': '2023-05-01', 'gbp_thb': 42.761, 'id': '670a'},
 {'date': '2023-05-02', 'gbp_thb': 42.477, 'id': 'd9f2'},
 {'date': '2023-05-03', 'gbp_thb': 42.63, 'id': 'f682'},
 {'date': '2023-05-04', 'gbp_thb': 42.456, 'id': '0734'},
 {'date': '2023-05-05', 'gbp_thb': 42.794, 'id': '21c0'},
 {'date': '2023-05-06', 'gbp_thb': 42.794, 'id': '13b7'},
 {'date': '2023-05-07', 'gbp_thb': 42.794, 'id': 'b665'},
 {'date': '2023-05-08', 'gbp_thb': 42.746, 'id': 'cc82'},
 {'date': '2023-05-09', 'gbp_thb': 42.539, 'id': 'bf71'},
 {'date': '2023-05-10', 'gbp_thb': 42.48, 'id': '50ff'},
 {'date': '2023-05-11', 'gbp_thb': 42.309, 'id': 'ac43'},
 {'date': '2023-05-12', 'gbp_thb': 42.267, 'id': '7384'},
 {'date': '2023-05-13', 'gbp_thb': 42.267, 'id': 'aab0'},
 {'date': '2023-05-14', 'gbp_thb': 42.267, 'id': 'b5d0'},
 {'date': '2023-05-15', 'gbp_thb': 42.329, 'id': '3989'},
 {'date': '2023-05-16', 'gbp_thb': 42.574, 'id': 'a818'},
 {'date': '2023-05-17', 'gbp_thb': 42.752, 'id': '7816'},
 {'date': '2023-

1.2.3 Convert to Pandas

In [16]:
conversion_rate = pd.DataFrame(result_conversion_rate)

conversion_rate

,date,gbp_thb,id
0,2023-05-01,42.761,670a
1,2023-05-02,42.477,d9f2
2,2023-05-03,42.630,f682
3,2023-05-04,42.456,0734
4,2023-05-05,42.794,21c0
...,...,...,...
384,2024-05-19,45.957,331a
385,2024-05-20,45.859,c838
386,2024-05-21,46.246,c9b0
387,2024-05-22,46.382,d631


1.2.4 Drop column that no need to show (column id)

In [17]:
conversion_rate = conversion_rate.drop(columns=['id'])

1.2.5 Change type of column date from string to dt.date same as merged_transaction

In [18]:
conversion_rate['date'] = pd.to_datetime(conversion_rate['date'])

conversion_rate.head()

,date,gbp_thb
0,2023-05-01,42.761
1,2023-05-02,42.477
2,2023-05-03,42.630
3,2023-05-04,42.456
4,2023-05-05,42.794


**1.3 Join the data**

1.3.1 Create finalDF from merge DataFrame merged_transaction with conversion_rate


In [19]:
final_df = merged_transaction.merge(conversion_rate, how="left", left_on="Date", right_on="date")

final_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77
...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63


1.3.2 For now we have column Price and Quantity but we still don’t have total amount. So, it’s from Price * Quantity

In [20]:
final_df["total_amount"] = final_df["Price"] * final_df["Quantity"]

final_df.head()

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,total_amount
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,257.64
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,383.40
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,138.36
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,127.80
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,71.64


1.3.3 After we have total_amount. Then we need the currency conversion. So, it’s from  (total_amount * gbp_thb)

In [21]:
final_df["thb_amount"] = final_df["total_amount"] * final_df["gbp_thb"]

final_df

,TransactionNo,Date,ProductNo,Price,Quantity,CustomerNo,ProductName,Country,Name,date,gbp_thb,total_amount,thb_amount
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,2024-05-10,45.77,257.64,11792.1828
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,2024-05-10,45.77,383.40,17548.2180
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,2024-05-10,45.77,138.36,6332.7372
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,2024-05-10,45.77,127.80,5849.4060
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,2024-05-10,45.77,71.64,3278.9628
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,2023-05-03,42.63,-37.92,-1616.5296
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,2023-05-03,42.63,-42.27,-1801.9701
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,2023-05-03,42.63,-11.74,-500.4762
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,2023-05-03,42.63,-65.40,-2788.0020


1.3.4 Delete Column no need to show and change the Column name

We can drop the column no need to use such as duplicated date with Date and column gpb_thb

In [22]:
final_df = final_df.drop(["date", "gbp_thb"], axis=1)

final_df.columns

Index(['TransactionNo', 'Date', 'ProductNo', 'Price', 'Quantity', 'CustomerNo',
       'ProductName', 'Country', 'Name', 'total_amount', 'thb_amount'],
      dtype='object')

1.3.5 Change the column name to lower alphabet and change the column name ending with No to _id

In [23]:
final_df.columns = ['transaction_id', 'date', 'product_id', 'price', 'quantity', 'customer_id',
       'product_name', 'customer_country', 'customer_name', 'total_amount','thb_amount']

final_df

,transaction_id,date,product_id,price,quantity,customer_id,product_name,customer_country,customer_name,total_amount,thb_amount
0,581482,2024-05-10,22485,21.47,12,17490.0,Set Of 2 Wooden Market Crates,United Kingdom,Sara Griffin,257.64,11792.1828
1,581475,2024-05-10,22596,10.65,36,13069.0,Christmas Star Wish List Chalkboard,United Kingdom,Michael Holt,383.40,17548.2180
2,581475,2024-05-10,23235,11.53,12,13069.0,Storage Tin Vintage Leaf,United Kingdom,Michael Holt,138.36,6332.7372
3,581475,2024-05-10,23272,10.65,12,13069.0,Tree T-Light Holder Willie Winkie,United Kingdom,Michael Holt,127.80,5849.4060
4,581475,2024-05-10,23239,11.94,6,13069.0,Set Of 4 Knick Knack Tins Poppies,United Kingdom,Michael Holt,71.64,3278.9628
...,...,...,...,...,...,...,...,...,...,...,...
536345,C536548,2023-05-03,22168,18.96,-2,12472.0,Organiser Wood Antique White,Germany,Stephen Pena,-37.92,-1616.5296
536346,C536548,2023-05-03,21218,14.09,-3,12472.0,Red Spotty Biscuit Tin,Germany,Stephen Pena,-42.27,-1801.9701
536347,C536548,2023-05-03,20957,11.74,-1,12472.0,Porcelain Hanging Bell Small,Germany,Stephen Pena,-11.74,-500.4762
536348,C536548,2023-05-03,22580,16.35,-4,12472.0,Advent Calendar Gingham Sack,Germany,Stephen Pena,-65.40,-2788.0020


**1.4 Output file**

1.4.1 Last step is Output to Parquet file with coding to_parque

Normally, pandas will save index (0,1,2,3) if we no need we can coding by index=False

In [24]:
final_df.to_parquet("output.parquet", index=False)